In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.utils import resample
import scipy.stats as st

import random
import seaborn as sns
import matplotlib.pylab as plt

import statsmodels.api as sm
import math
from math import log

In [2]:
df = pd.read_csv('AB_test_data.csv')

# 1

In [3]:
df.loc[df['purchase_TF']==False,'tf']=0
df.loc[df['purchase_TF']==True,'tf']=1

In [4]:
df.drop(columns=['purchase_TF','date','id'],inplace=True)

In [5]:
df_A = df[df.Variant=='A']
df_B = df[df.Variant=='B']

In [6]:
size_A = df_A.shape[0]
size_B = df_B.shape[0]

In [7]:
con_rate_A = df_A.tf.mean()
con_rate_B = df_B.tf.mean()

In [8]:
con_rate_A

0.149616

In [9]:
con_rate_B

0.1766

In [10]:
con_rate_A-con_rate_B

-0.026984000000000008

In [11]:
std_A = np.std(df_A.tf)

H0:μ1<=μ0 ; H1:μ1>μ0

In [12]:
z = (con_rate_B-con_rate_A)/(math.sqrt(con_rate_A*(1-con_rate_A)/size_B))
z

5.349273094732516

In [13]:
st.norm.ppf(0.95)

1.6448536269514722

z is bigger than 1.6448, so reject H0.  Alternative B improved conversion rates over alternative A.

# 2

In [14]:
delta = con_rate_B-con_rate_A
con_rate_avg = (con_rate_A+con_rate_B)/2
sample_size = (st.norm.ppf(0.05)*\
     math.sqrt(2*con_rate_avg*(1-con_rate_avg))+\
     st.norm.ppf(0.2)*math.sqrt(con_rate_A*(1-con_rate_A)+con_rate_B*(1-con_rate_B)))**2/\
    (delta)**2
sample_size = int(sample_size)
sample_size

2317

In [15]:
# sample_size = (con_rate_A*(1-con_rate_A)+con_rate_B*(1-con_rate_B))*\
#     ((st.norm.ppf(0.05)+st.norm.ppf(0.2))**2)/(delta)**2
# sample_size = int(sample_size)
# sample_size

In [31]:
#p-value
sample=[]
for i in range(10):
    s = df_B.sample(n=sample_size,axis=0)
    sample.append(s)
    pvalue = sm.stats.ztest(df_A.tf, sample[i].tf,alternative='smaller')[1]
    print(i+1,") ",pvalue)

1 )  2.5240484182121445e-05
2 )  0.0004747017670236977
3 )  2.4245054256464644e-06
4 )  4.1031749584733985e-05
5 )  3.1854872636766513e-06
6 )  0.0005820553450660916
7 )  0.00025275472469747423
8 )  4.172065503287491e-06
9 )  1.800992029764702e-07
10 )  0.0012755815648992383


In [32]:
#z-score
z_score = []
for i in range(10):
    sample_mean = sample[i].tf.mean()
    z = (sample_mean-con_rate_A)/(math.sqrt(con_rate_A*(1-con_rate_A)/sample_size))
    z_score.append(z)
    print(i+1,") ",z)

1 )  4.096765520541687
2 )  3.3396122974436118
3 )  4.620948521148044
4 )  3.980280409295828
5 )  4.562705965525119
6 )  3.281369741820682
7 )  3.5143399643123963
8 )  4.504463409902189
9 )  5.145131521754405
10 )  3.048399519328968


All of the z-scores are larger than 1.6448, so reject H0. 

Alternative B improved conversion rates over alternative A.

# 3

H0: The conversion rate of Varient B is 0.149616

H1: The conversion rate of Varient B is 0.1766

In [20]:
ln_A = log(1/0.05)
ln_B = log(0.2)

In [33]:
iteration = []
for i in range(10):
    lambda_n = 0
    n = 0
    for j in range(sample_size):
        mean_A = df_A.tf.mean()
        mean_sample = sample[i].tf.mean()
        lambda_0 = log((1-mean_sample)/(1-mean_A))
        lambda_1 = log((mean_sample)/(mean_A))
        if sample[i].iloc[j,1] == 1:
            lambda_n = lambda_n + lambda_1
        else:
            lambda_n = lambda_n + lambda_0
        if lambda_n > ln_B and lambda_n < ln_A:
            n = n+1
        elif lambda_n <= ln_B:
            print('For sample',i+1,', accept H0 and itertaion is ',n+1,'. ln(lambda_n) is',lambda_n)
            iteration.append(n)
            break
        elif lambda_n >= ln_A:
            print('For sample',i+1,', accept H1 and itertaion is ',n+1,'. ln(lambda_n) is',lambda_n)
            iteration.append(n)
            break

For sample 1 , accept H1 and itertaion is  604 . ln(lambda_n) is 3.026851602807017
For sample 2 , accept H1 and itertaion is  996 . ln(lambda_n) is 3.0881627700134784
For sample 3 , accept H1 and itertaion is  606 . ln(lambda_n) is 3.0262389104746967
For sample 4 , accept H1 and itertaion is  430 . ln(lambda_n) is 3.1158223529584452
For sample 5 , accept H1 and itertaion is  1045 . ln(lambda_n) is 3.0454422075062877
For sample 6 , accept H1 and itertaion is  1822 . ln(lambda_n) is 2.998749833154736
For sample 7 , accept H1 and itertaion is  425 . ln(lambda_n) is 3.06484346658622
For sample 8 , accept H1 and itertaion is  747 . ln(lambda_n) is 3.163888224254192
For sample 9 , accept H1 and itertaion is  343 . ln(lambda_n) is 3.0944165326526285
For sample 10 , accept H1 and itertaion is  1724 . ln(lambda_n) is 3.008797006936828


In [34]:
np.mean(iteration)

873.2

In [35]:
iteration

[603, 995, 605, 429, 1044, 1821, 424, 746, 342, 1723]